In [1]:
import pandas as pd
import pandas as pd
import geopandas as gpd
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256 
from bokeh.palettes import BrBG, PiYG, RdGy, RdYlGn, YlGnBu
from bokeh.models import LinearColorMapper, ColorBar
#import streamlit as st
#from streamlit_folium import st_folium
#import folium
#from leaflet import Map, GeoJSON

In [2]:
def format_df(filename):
    df = pd.read_csv(filename)
    df['harvest'] = pd.to_datetime(df['harvest'], format='%Y/%m/%d')
    df['planting'] = pd.to_datetime(df['planting'], format='%Y/%m/%d')
    df['date'] = df['harvest'].dt.strftime("%Y")
    df = df.rename(columns={'cname': 'Region'})
    #df = df.rename(columns={df.columns[1]: "Region"})
    #df['Region'] = df['Region'].str.upper()
    return df

In [3]:
unimodal=format_df('/Users/gamoyo/Documents/GitHub/yieldprediction/rheas/tanzania_tamsat_25km_districts_unimod_dssatTable_2021_2023_100kg.csv')

print(unimodal)

          id           Region   planting    harvest   wsgd  gwad  date
0      21665            Nyasa 2023-02-18 2023-06-22  0.166  3063  2023
1      21666            Nyasa 2023-01-22 2023-06-02  0.000  3297  2023
2      21667            Nyasa 2023-02-16 2023-06-22  0.311  3194  2023
3      23176           Uvinza 2023-02-04 2023-06-16  0.000  3635  2023
4      23177           Uvinza 2023-02-11 2023-06-22  0.000  3664  2023
...      ...              ...        ...        ...    ...   ...   ...
13195  23362  Lake Tanganyika 2022-02-18 2022-06-22  0.000  2673  2022
13196  23363  Lake Tanganyika 2022-02-12 2022-06-22  0.000  3267  2022
13197  23364  Lake Tanganyika 2022-02-03 2022-06-22  0.000  4059  2022
13198  23365  Lake Tanganyika 2022-02-09 2022-06-22  0.000  3366  2022
13199  23366  Lake Tanganyika 2022-02-19 2022-06-22  0.000  2574  2022

[13200 rows x 7 columns]


In [4]:
# Unimodal season:  October--August (LR)
lrStart = [9, 10, 11, 12, 1,2, 3, 4]
lrEnd = [2, 3, 4, 5, 6, 7, 8]
def LR_metrics(lr, lrStart, lrEnd, season):
    lr.loc[(pd.to_datetime(lr['planting']).dt.month.isin(lrStart)) & (pd.to_datetime(lr['harvest']).dt.month.isin(lrEnd)), 'Season'] = season
    lr = lr.groupby(['Region', 'date', 'Season'], as_index=False).agg({'gwad': 'mean'})

    return lr

rh_lr = LR_metrics(unimodal, lrStart, lrEnd, "LR")
print(rh_lr.head())

     Region  date Season         gwad
0      Bahi  2022     LR  3783.650000
1      Bahi  2023     LR  3123.037500
2   Buhigwe  2022     LR  3725.214286
3   Buhigwe  2023     LR  2691.573529
4  Chamwino  2022     LR  3743.050000


In [5]:
rh_lr.query('date in [2022]')

,Region,date,Season,gwad


In [14]:
shapefile = gpd.read_file("/Users/gamoyo/Documents/GitHub/yieldprediction/rheas/gadm36_TZA_2_LongRain.shp")
shapefile.rename(columns={'NAME_2':'Region'}, inplace=True)
country_shapefile = gpd.read_file('tanzania.shp')
#shapefile["NewDist20"] = shapefile.index
#print(shapefile)
merged_data = pd.merge(shapefile, rh_lr, on="Region")

In [7]:
outfp = "merged_unimodal.shp"
merged_data.to_file(outfp)

In [15]:
geo_source = GeoJSONDataSource(geojson=merged_data.to_json())
country_geo_source = GeoJSONDataSource(geojson=country_shapefile.to_json())

In [16]:
color_mapper = LinearColorMapper(palette=Viridis256, low=min(merged_data['gwad']), high=max(merged_data['gwad']))

In [17]:
p1 = figure(title='Yield Data by District',
           plot_width=650, plot_height=600)

p1.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p1.patches('xs','ys', source=geo_source,
          fill_color={'field': 'gwad', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@Region'), ('gwad', '@gwad')])
p1.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p1.add_layout(color_bar, 'right')

In [18]:
show(p1)

In [12]:
color_mapper1 = LinearColorMapper(palette=Viridis256, low=min(merged_data['gwad']), high=max(merged_data['gwad']))

p2 = figure(title='Yield Data by District',
           plot_width=650, plot_height=600)

p2.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p2.patches('xs','ys', source=geo_source,
          fill_color={'field': 'gwad', 'transform': color_mapper1},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@gwad'), ('gwad', '@gwad')])
p2.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper1, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p2.add_layout(color_bar, 'right')

In [13]:
from bokeh.layouts import column
layout = column(p1, p2)
show(layout)

RuntimeError: Models must be owned by only a single document, MultiLine(id='1041', ...) is already in a doc

In [ ]:
#from bokeh.io import output_file, show
#output_file("plot.html")
#show(layout)

In [ ]:
#from bokeh.io import save

# specify the output file name and location
#save(p, filename='yield_map.html', title='Yield Data Map')